## Домашнее задание к занятию "Алгоритмы классификации: линейные методы, логистическая регрессия и SVM"

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error

In [2]:
"""
Загружаем файл """

data = pd.read_csv('adult.csv' )
data.head()

,age,workclass,fnlwgt,education,educational-num,marital-status,occupation,relationship,race,gender,capital-gain,capital-loss,hours-per-week,native-country,income
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K


In [3]:
"""
Проверяем на наличие нулевых значений """

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 48842 entries, 0 to 48841
Data columns (total 15 columns):
age                48842 non-null int64
workclass          48842 non-null object
fnlwgt             48842 non-null int64
education          48842 non-null object
educational-num    48842 non-null int64
marital-status     48842 non-null object
occupation         48842 non-null object
relationship       48842 non-null object
race               48842 non-null object
gender             48842 non-null object
capital-gain       48842 non-null int64
capital-loss       48842 non-null int64
hours-per-week     48842 non-null int64
native-country     48842 non-null object
income             48842 non-null object
dtypes: int64(6), object(9)
memory usage: 5.6+ MB


In [4]:
"""
Нулевых значений нет


Преобразуем таргет (income) в бинарную переменную """

from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
le.fit(data['income'])
y = pd.Series(data = le.transform(data['income']))

In [5]:
"""
Проверяем принадлежность классов бинарным значениям """

le.transform(['<=50K','>50K'])

array([0, 1], dtype=int64)

In [6]:
"""
Таким образом, 0 - это бедные, 1 - богатые 


Решение задачи начнем с проверки расистской гипотезы: 
Уровень дохода зависит от цвета кожи (и скорее всего белые богаче, черные беднее)

Для этого пока рассмотрим только одну переменную - 'race'
"""

X1 = pd.get_dummies(data['race'], columns = ['race'])

In [7]:
"""
Пока имеем только 5 столбцов - по числу рас в датасете  """

X1.shape

(48842, 5)

In [8]:
"""
Делим всю выборку на трейн и тест, обучаем модель логистической регрессии и проверяем результат"""

X_train, X_test, y_train, y_test = train_test_split(X1, y, test_size=0.2, random_state=42)

In [9]:
model = LogisticRegression()

In [10]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\rusla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [11]:
""" 
Для проверки результата сделаем самую простую метрику, которая просто будет считать отношение 
правильно предсказанных результатов к общему их числу """

def check_result(y,y_pred):
    count=0
    y=list(y)
    y_pred=list(y_pred)
    for i in range(len(y)):
        if y[i]==y_pred[i]:
            count+=1
    return count/len(y)

In [12]:
"""
Проверим своей метрикой  """

check_result(y_test,y_pred)

0.7655850138192241

In [13]:
"""
И методом score   """

model.score(X_test, y_test)

0.7655850138192241

In [15]:
""" 
Поскольку результат моей метрики полностью совпадает с результатом метода score, 
делаю вывод, что score именно так и работает.

Посмотрим еще значение MSE """

mean_squared_error(y_test, y_pred)

0.2344149861807759

In [16]:
"""
Посмотрим на размеры коэффициентов модели  """

pd.DataFrame(columns=X1.columns, data=model.coef_)

,Amer-Indian-Eskimo,Asian-Pac-Islander,Black,Other,White
0,-0.619008,0.367058,-0.642597,-0.666154,0.242708


In [17]:
"""
В целом гипотеза подтвердилась: у белых положительный коэффициент, который в целом склоняет модель к единичке.
У черных - сильно отрицательный, что больше приближает их к нулю.
Немного удивили азиаты, у них вес даже выше, чем у белых
Индейцы/эскимосы в высокие заработки тоже особо не лезут.

Расистская модель показала неплохой результат - 76,5%.
Но в датасете есть еще много информации, которая наверняка позволит улучшить результат

Добавляем новые признаки, те, что категориальные, сразу прогоняем через get_dummies """

SelectedColumns = data[['age', 'workclass', 'education', 
       'marital-status', 'occupation', 'race', 'gender',
       'hours-per-week', 'native-country']]
X2 = pd.get_dummies(SelectedColumns, columns = ['workclass', 'education', 
       'marital-status', 'occupation', 'race', 'gender',
        'native-country'])

In [18]:
"""
Матрица Х раздулась до 98 признаков  """

X2.shape

(48842, 98)

In [19]:
"""
Делим новую выборку на трейн и тест """

X_train, X_test, y_train, y_test = train_test_split(X2, y, test_size=0.2, random_state=42)

In [20]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\rusla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [21]:
model.score(X_test, y_test)

0.8396970007165524

In [22]:
check_result(y_test,y_pred)

0.8396970007165524

In [23]:
mean_squared_error(y_test, y_pred)

0.16030299928344763

In [25]:
"""
Результат модели после добавления признаков улучшился до 84%, MSE также уменьшился

Из неиспользованных признаков остались следующие:

"""
set(data.columns)-set(SelectedColumns.columns)-{'income'}

{'capital-gain', 'capital-loss', 'educational-num', 'fnlwgt', 'relationship'}

In [26]:
"""
В результате их перебора, подробности которого я опускаю, выяснилось, 
что улучшить модель может только добавление данных из колонки 'relationship'   

Добавляем их в матрицу Х  """

SelectedColumns = data[['age', 'workclass',  'education', 
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
        'hours-per-week', 'native-country']]
X3 = pd.get_dummies(SelectedColumns, columns = ['workclass', 'education', 
       'marital-status', 'occupation', 'relationship', 'race', 'gender',
        'native-country'])

In [27]:
"""
Матрица выросла еще на 6 столбцов - по числу категорий в колонке 'relationship' """

X3.shape

(48842, 104)

In [28]:
"""
Делим новую выборку на трейн и тест """

X_train, X_test, y_train, y_test = train_test_split(X3, y, test_size=0.2, random_state=42)

In [29]:
model.fit(X_train, y_train)
y_pred = model.predict(X_test)

C:\Users\rusla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [30]:
model.score(X_test, y_test)

0.8426655747773569

In [31]:
mean_squared_error(y_test, y_pred)

0.15733442522264304

In [32]:
"""
Добавление признака 'relationship' улучшило результат модели на 0,29% и уменьшило значение MSE.

В качестве последнего штриха, попробую разные значения коэффициента регуляризации.
Многочисленные эксперименты также опускаю. Оставляю лучший полученный результат с коэффициентом С=10
"""

model_C=LogisticRegression(C=10)
model_C.fit(X_train, y_train)
y_pred = model_C.predict(X_test)

C:\Users\rusla\Anaconda3\lib\site-packages\sklearn\linear_model\logistic.py:433: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)


In [33]:
model_C.score(X_test, y_test)

0.8430750332685024

In [34]:
mean_squared_error(y_test, y_pred)

0.15692496673149758

Изменение коэффициента регуляризации улучшило результат еще на 0,04% и немного уменьшило MSE


Всё!